In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
from tensorflow import keras
import time

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
start = time.time()

char_dict = {"char": [], "matrix": []}
char_set = dict()
src = r"dataset/handwritten_letters"
for i in range(500):
    for folder in os.listdir(src):
        char_dict["char"].append(folder)
        if folder not in char_set:
            char_set[folder] = sorted_alphanumeric(os.listdir(src + '/' + folder))
        img = cv2.imread(f"{src}/{folder}/{char_set[folder][i]}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if not img.shape == (32, 32):
            img = cv2.resize(img, (32, 32))
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        img = cv2.bitwise_not(img)
        img = img.flatten()
        char_dict["matrix"].append(img)


stop = time.time()
print(stop-start, 'seconds')

8.766955852508545 seconds


In [4]:
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
y = np.array(char_dict['char'])

y = le.fit_transform(y)
X = np.array(char_dict["matrix"])

/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [5]:
split = int(round(0.9 * len(X)))

train_X = X[:split]/255
train_y = y[:split]
test_X = X[split:]/255
test_y = y[split:]

In [6]:
train_X = train_X.astype('int32')
test_X = test_X.astype('int32')

In [7]:
values = []
for char in train_X:
    for value in char:
        if value not in values:
            values.append(value)
print('x values:', values)
values = []
for char in train_y:
    for value in char:
        if value not in values:
            values.append(value)
print('y values:', values)

x values: [0, 1]
y values: [1, 0]


In [8]:
train_X = tf.constant(train_X)
train_y = tf.constant(train_y)

test_X = tf.constant(test_X)
test_y = tf.constant(test_y)

In [9]:
train_X

<tf.Tensor: id=0, shape=(15750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [10]:
train_y

<tf.Tensor: id=1, shape=(15750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [11]:
test_X

<tf.Tensor: id=2, shape=(1750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [12]:
test_y

<tf.Tensor: id=3, shape=(1750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [13]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=1024))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(35, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model.fit(train_X, train_y, epochs=20)

Train on 15750 samples
Epoch 1/20
15750/15750 [==============================] - 5s 339us/sample - loss: 1.3570 - accuracy: 0.6250
Epoch 2/20
15750/15750 [==============================] - 5s 302us/sample - loss: 0.6035 - accuracy: 0.8180
Epoch 3/20
15750/15750 [==============================] - 5s 286us/sample - loss: 0.4087 - accuracy: 0.8737
Epoch 4/20
15750/15750 [==============================] - 5s 308us/sample - loss: 0.2973 - accuracy: 0.9055
Epoch 5/20
15750/15750 [==============================] - 5s 312us/sample - loss: 0.2249 - accuracy: 0.9282
Epoch 6/20
15750/15750 [==============================] - 5s 338us/sample - loss: 0.1735 - accuracy: 0.9429
Epoch 7/20
15750/15750 [==============================] - 5s 313us/sample - loss: 0.1387 - accuracy: 0.9516
Epoch 8/20
15750/15750 [==============================] - 5s 314us/sample - loss: 0.1177 - accuracy: 0.9591
Epoch 9/20
15750/15750 [==============================] - 5s 307us/sample - loss: 0.0990 - accuracy: 0.9665
Epoch

In [15]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.9800000190734863% 
 Error on test data: 0.019999980926513672

Accuracy on test data: 0.8320000171661377% 
 Error on test data: 0.1679999828338623


In [27]:
def get_wrong_predictions(preds):
    in_test_y = char_dict['char'][split:]
    chars = char_dict['char'][:35]
    print("{Actual letter} - {Predicted letter}")
    for i, prediction in enumerate(preds):
        prediction = list(prediction)
        actual_letter = in_test_y[i]
        predicted_letter = chars[prediction.index(max(prediction))]
        if predicted_letter != actual_letter:
            print(actual_letter, "-", predicted_letter)

In [28]:
get_wrong_predictions(pred_test)

{Actual letter} - {Predicted letter}
G - Q
J - S
U - K
9 - Q
N - R
G - S
S - J
V - G
N - 4
G - 9
Z - 2
T - R
S - J
H - W
J - 5
4 - Y
B - C
9 - Q
6 - G
D - G
V - N
4 - 9
X - A
6 - W
8 - B
J - G
D - Q
9 - Q
T - Y
S - J
2 - A
9 - Q
I - 1
G - 7
S - J
F - T
H - 4
4 - A
P - Q
R - A
I - 1
2 - G
N - W
S - J
F - 8
H - R
M - R
B - G
2 - Z
I - J
J - S
D - G
2 - G
N - H
T - R
E - 8
U - W
7 - T
N - M
G - A
1 - I
T - R
A - B
C - 0
D - B
2 - S
N - H
G - P
A - H
Q - A
L - I
N - W
6 - B
Q - A
P - Q
9 - Q
6 - E
8 - R
S - I
C - U
L - E
1 - I
T - F
H - M
Q - Y
B - D
K - W
2 - D
P - G
6 - G
8 - G
A - D
4 - R
B - 8
2 - E
U - V
I - S
N - R
8 - P
S - J
Q - 9
2 - B
P - F
J - Y
3 - 5
P - 9
N - A
G - 9
H - M
4 - U
B - D
L - H
2 - D
P - 7
I - L
G - Q
8 - Q
3 - J
2 - D
T - 4
Q - E
2 - A
9 - Q
8 - Q
H - A
3 - 5
B - 8
P - D
V - Y
Q - A
6 - G
Z - A
4 - 9
3 - 5
B - R
2 - J
T - R
F - P
H - N
Q - G
2 - S
W - N
J - U
5 - F
7 - 9
G - Q
6 - G
1 - I
J - Y
P - D
7 - 9
I - 1
N - W
8 - A
H - L
Y - 4
5 - F
P - Q
U - W
7 - 9
I -